In [73]:
%run utility.ipynb
import pandas as pd
import altair as alt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans

#DataFrame 1
df = loadAndCleanData("Career_Stats_Passing.csv")
df = df.replace("--",0.0)
print(df)


# This data is coming from Kaggle under the portfolio labeled : "NFL-Statistics-Scrape"
#The primary player I am looking at are:
    #Biggest draft bust : Johnny Manziel (2420)
    #Biggest draft success : Ryan Fitzpatrick ()
    

                 Player Id            Name Position  Year  \
0        tomfarris/2513861     Farris, Tom        0  1948   
1        tomfarris/2513861     Farris, Tom        0  1947   
2        tomfarris/2513861     Farris, Tom        0  1946   
3       billdemory/2512778    Demory, Bill        0  1974   
4       billdemory/2512778    Demory, Bill        0  1973   
...                    ...             ...      ...   ...   
8520  billanderson/2508534  Anderson, Bill        0  1962   
8521  billanderson/2508534  Anderson, Bill        0  1961   
8522  billanderson/2508534  Anderson, Bill        0  1960   
8523  billanderson/2508534  Anderson, Bill        0  1959   
8524  billanderson/2508534  Anderson, Bill        0  1958   

                     Team  Games Played Passes Attempted Passes Completed  \
0         Chicago Rockets             0                0                0   
1           Chicago Bears             9                2                0   
2           Chicago Bears           

In [115]:
#DataFrame 2
ogDF = df.head(100)
x = df.loc[df["Name"]=="Fitzpatrick, Ryan"]
y = df.loc[df["Name"]=='Manziel, Johnny']
z = x.append(y)
z = z.append(ogDF)



z["TD Passes"] = z["TD Passes"].astype(float)
z["Passes Completed"] = z["Passes Completed"].astype(float)
#z["Passing Yards"] = z["Passing Yards"].apply(lambda x:x.replace(",",""))
#z["Passing Yards"] = z["Passing Yards"].astype(float)


z = z.sort_values(by=["Passes Completed"], ascending=True)
z = z.sort_values(by=["TD Passes"], ascending=True)

pd.set_option('display.max_columns', None)
print(z)

#I created this dataframe to represent the first
    #100 players of the dataset as well as the additional
    #playerz I wished to analyze (Fitzpatrick and Manziel)
        

                    Player Id               Name Position  Year  \
42        paulmaguire/2519977      Maguire, Paul        0  1966   
6          breezyreid/2523928       Reid, Breezy        0  1955   
5          breezyreid/2523928       Reid, Breezy        0  1956   
0           tomfarris/2513861        Farris, Tom        0  1948   
1           tomfarris/2513861        Farris, Tom        0  1947   
...                       ...                ...      ...   ...   
7409  ryanfitzpatrick/2506581  Fitzpatrick, Ryan       QB  2011   
86      frantarkenton/2527019    Tarkenton, Fran        0  1978   
89      frantarkenton/2527019    Tarkenton, Fran        0  1975   
97      frantarkenton/2527019    Tarkenton, Fran        0  1967   
7405  ryanfitzpatrick/2506581  Fitzpatrick, Ryan       QB  2015   

                   Team  Games Played Passes Attempted  Passes Completed  \
42        Buffalo Bills            14                0               0.0   
6     Green Bay Packers            12      

In [54]:
# Visualization 1
alt.Chart(z).mark_circle(size = 100).encode(
    x = "Games Played",
    y = "Name",
    color=alt.Color('Name', legend=alt.Legend(title="Player Names")),
    tooltip = ["Name","Team", "Games Played"]
).interactive()

alt.Chart(...)

In [55]:
# Visualization 2
alt.Chart(z).mark_bar().encode(
    x = "Passes Completed",
    y = "Name",
    color = "Passes Completed",
    tooltip = ["Name", "Team", "Completion Percentage", "Passes Completed"]
)

alt.Chart(...)

In [56]:
# Visualization 3
alt.Chart(z).mark_bar().encode(
    x = "TD Passes",
    y = "Name",
    tooltip = ["Name", "Team", "TD Passes", "Sacks"]
).configure_mark(opacity = 0.6,color = 'red').properties(width=525)

alt.Chart(...)

In [100]:
# Visualization 4
alt.Chart(z).mark_area().encode(
x = "Year",
y = "TD Passes",
tooltip = ["Name", "Team", "Completion Percentage", "TD Passes"],
color=alt.Color('Name', legend=alt.Legend(title="Player Names"))
).configure_mark(opacity = 0.6, color = 'red').properties(width=650,height = 500)

alt.Chart(...)

In [122]:
def playerAVG(data):
    h = data.copy()
    h["Passing Yards"] = h["Passing Yards"].astype(str)
    h["Passing Yards"] = h["Passing Yards"].apply(lambda x:x.replace(",",""))
    h["Passing Yards"] = h["Passing Yards"].astype(float)
    avg1 = h["Passing Yards"].sum() / len(h["Passing Yards"])
    
    h["Completion Percentage"] = h["Completion Percentage"].astype(str)
    h["Completion Percentage"] = h["Completion Percentage"].apply(lambda x:x.replace(",",""))
    h["Completion Percentage"] = h["Completion Percentage"].astype(float)
    avg2 = h["Completion Percentage"].sum() / len(h["Completion Percentage"])
    
    h["TD Passes"] = h["TD Passes"].astype(str)
    h["TD Passes"] = h["TD Passes"].apply(lambda x:x.replace(",",""))
    h["TD Passes"] = h["TD Passes"].astype(float)
    avg3 = h["TD Passes"].sum() / len(h["TD Passes"])
    

    print("Average Completion Percentage = " + str(avg1))
    print("Average Passing Yards Per Game = " + str(avg2))
    print("Average Touch Down Passes Per Season = " + str(avg3))

In [127]:
#Successes
print("Successful Players")
print("Fitzpatrick")
playerAVG(z.loc[z["Name"]=="Fitzpatrick, Ryan"])
print()
print("Tarkenton")
playerAVG(z.loc[z["Name"]=="Tarkenton, Fran"])
print()

#Busts
print("Bust Players")
print("Manziel")
playerAVG(z.loc[z["Name"]=="Manziel, Johnny"])
print()
print("Scarbath")
playerAVG(z.loc[z["Name"]=="Scarbath, Jack"])

Successful Players
Fitzpatrick
Average Completion Percentage = 2157.3333333333335
Average Passing Yards Per Game = 49.44166666666667
Average Touch Down Passes Per Season = 13.833333333333334

Tarkenton
Average Completion Percentage = 2685.285714285714
Average Passing Yards Per Game = 57.43571428571429
Average Touch Down Passes Per Season = 18.928571428571427

Bust Players
Manziel
Average Completion Percentage = 837.5
Average Passing Yards Per Game = 54.599999999999994
Average Touch Down Passes Per Season = 3.5

Scarbath
Average Completion Percentage = 622.6666666666666
Average Passing Yards Per Game = 34.86666666666667
Average Touch Down Passes Per Season = 6.0


In [58]:
def runKNN(df, x, y, k): 
    # Let's assume that we're using numeric features to predict a categorical label
    X = x.values
    Y = y.values.reshape(-1, 1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)
    
    # Build a kNN classifier
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    
    # Compute the quality of our predictions
    score = knn.score(X_test, Y_test)
    print("Predicts " + y.name + " with " + str(score) + " accuracy")
    print("F1 score is " + str(f1_score(Y_test, knn.predict(X_test), labels=y.unique(), average='weighted')))
    print("Chance is: " + str(1.0 / len(y.unique())))
    
    return knn

In [99]:
z["Position"] = z["Position"].astype(str)
z["TD Passes"] = z["TD Passes"].astype(float)

x = z[["Passes Attempted"]]
y = z["Position"]


runKNN(z, x, y, 1)

for k in range(1, 5): 
    print(str(k) + " Neighbors:")
    runKNN(z, x, y, k)
    
y.value_counts()


Predicts Position with 0.8260869565217391 accuracy
F1 score is 0.7867494824016563
Chance is: 0.5
1 Neighbors:
Predicts Position with 0.8260869565217391 accuracy
F1 score is 0.7867494824016563
Chance is: 0.5
2 Neighbors:
Predicts Position with 0.8695652173913043 accuracy
F1 score is 0.8088978766430739
Chance is: 0.5
3 Neighbors:
Predicts Position with 0.8695652173913043 accuracy
F1 score is 0.8088978766430739
Chance is: 0.5
4 Neighbors:
Predicts Position with 0.8695652173913043 accuracy
F1 score is 0.8088978766430739
Chance is: 0.5


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':
/anacond

0     100
QB     14
Name: Position, dtype: int64

In [94]:

player = df[(df["Name"]=="Fitzpatrick, Ryan") & (df["Year"]==2011)]
print(player)
playerData = player.select_dtypes(include="number")

model = runKNN(df, x, y, 3)
print("k-NN thinks " + player["Name"] + " is a " + model.predict(playerData)["Year"])

relatedPlayers = model.kneighbors(playerData, n_neighbors=3, return_distance=False)

for p in relatedPlayers[0]: 
    print(df.iloc[int(p)]["Name"])
    

                    Player Id               Name Position  Year  \
7409  ryanfitzpatrick/2506581  Fitzpatrick, Ryan       QB  2011   

               Team  Games Played Passes Attempted Passes Completed  \
7409  Buffalo Bills            16              569              353   

     Completion Percentage  Pass Attempts Per Game Passing Yards  \
7409                  62.0                    35.6         3,832   

     Passing Yards Per Attempt Passing Yards Per Game TD Passes  \
7409                       6.7                  239.5        24   

     Percentage of TDs per Attempts Ints Int Rate Longest Pass  \
7409                            4.2   23      4.0          60T   

     Passes Longer than 20 Yards Passes Longer than 40 Yards Sacks  \
7409                          48                           8    22   

     Sacked Yards Lost  Passer Rating  Cluster  
7409               148           79.1        7  
Predicts Position with 0.8695652173913043 accuracy
F1 score is 0.8088978766430

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


ValueError: query data dimension must match training data dimension

In [86]:
player = df[df["Name"] == "Manziel, Johnny"]
relatedPlayers = model.kneighbors(player.select_dtypes(include="number"), n_neighbors=5, return_distance=False)

for p in relatedPlayers[0]: 
    print(df.iloc[int(p)]["Name"])

ValueError: query data dimension must match training data dimension

In [87]:
def runKMeans(df, k):

    X = df.select_dtypes(include="number")

    kmeans = KMeans(n_clusters=k)


    kmeans.fit(X)
    

    df["Cluster"] = pd.Series(kmeans.predict(X), index=df.index)
    return kmeans

In [88]:

runKMeans(df, 5)


MAX_K = 30

scores = []
for k in range(1, MAX_K):
    model = runKMeans(df, k)

    scores.append(model.inertia_)

scoresDf = pd.DataFrame()
scoresDf["k"] = range(1, MAX_K)
scoresDf["Score"] = scores

alt.Chart(scoresDf).mark_line().encode(
    x = "k",
    y = "Score"
)

alt.Chart(...)

In [89]:
runKMeans(df, 5)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [90]:
runKMeans(df, 8)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)